In [1]:
from janome.tokenizer import Tokenizer

t = Tokenizer()
sentences = [
    'おいしいビールを飲む', 'コーヒーを飲む', 'おいしいクラフトビールを買う'
]

words_list = []
for sentence in sentences:
    words_list.append(t.tokenize(sentence, wakati=True))
words_list

[['おいしい', 'ビール', 'を', '飲む'],
 ['コーヒー', 'を', '飲む'],
 ['おいしい', 'クラフト', 'ビール', 'を', '買う']]

In [2]:
unique_words = []
for words in words_list:
    for word in words:
        if word not in unique_words:
            unique_words.append(word)
unique_words

['おいしい', 'ビール', 'を', '飲む', 'コーヒー', 'クラフト', '買う']

In [3]:
bow_list = []
for words in words_list:
    bag_of_words = []
    for unique_word in unique_words:
        num = words.count(unique_word)
        bag_of_words.append(num)
    bow_list.append(bag_of_words)
bow_list

[[1, 1, 1, 1, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0], [1, 1, 1, 0, 0, 1, 1]]

In [4]:
from math import log
num_of_sentences = len(sentences)
idf = []
for i in range(len(unique_words)):
    count = 0
    for bow in bow_list:
        if bow[i] > 0:
            count += 1
    idf.append(log((num_of_sentences + 1) / (count + 1)))
idf

[0.28768207245178085,
 0.28768207245178085,
 0.0,
 0.28768207245178085,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453]

In [5]:
bow = bow_list[1]
num_of_words = sum(bow)
tfidf = []
for i, value in enumerate(bow):
    tf = value / num_of_words
    tfidf.append(tf * (idf[i] + 1))
tfidf

[0.0,
 0.0,
 0.3333333333333333,
 0.42922735748392693,
 0.5643823935199818,
 0.0,
 0.0]

In [6]:
from janome.tokenizer import Tokenizer

BEGIN = '__BEGIN__'
END = '__END__'

sentence = 'おいしいビールを飲もう'

t = Tokenizer()
words = t.tokenize(sentence, wakati=True)
words = [BEGIN] + words + [END]

three_words_list = []
for i in range(len(words) - 2):
    three_words_list.append(words[i:i+3])
three_words_list

[['__BEGIN__', 'おいしい', 'ビール'],
 ['おいしい', 'ビール', 'を'],
 ['ビール', 'を', '飲も'],
 ['を', '飲も', 'う'],
 ['飲も', 'う', '__END__']]

In [7]:
from collections import Counter

def get_three_words_list(sentence):
    """文章を３単語の組にして返す"""
    t = Tokenizer()
    words = t.tokenize(sentence, wakati=True)
    words = [BEGIN] + words + [END]
    three_words_list = []
    for i in range(len(words) - 2):
        three_words_list.append(tuple(words[i:i+3]))
    return three_words_list
    
sentences = ['おいしいビールを飲もう', 'ビールを飲もう', 'おいしいビールは生']
three_words_list = []
for sentence in sentences:
    three_words_list += get_three_words_list(sentence)
three_words_count = Counter(three_words_list)
three_words_count

Counter({('__BEGIN__', 'おいしい', 'ビール'): 2,
         ('おいしい', 'ビール', 'を'): 1,
         ('ビール', 'を', '飲も'): 2,
         ('を', '飲も', 'う'): 2,
         ('飲も', 'う', '__END__'): 2,
         ('__BEGIN__', 'ビール', 'を'): 1,
         ('おいしい', 'ビール', 'は'): 1,
         ('ビール', 'は', '生'): 1,
         ('は', '生', '__END__'): 1})

In [8]:
def generate_markov_dict(three_words_count):
    """マルコフ連鎖での文章生成用の辞書データを生成する"""
    markov_dict = {}
    for three_words, count in three_words_count.items():
        two_words = three_words[:2]
        next_word = three_words[2]
        if two_words not in markov_dict:
            markov_dict[two_words] = {'words': [], 'weights': []}
        markov_dict[two_words]['words'].append(next_word)
        markov_dict[two_words]['weights'].append(count)
    return markov_dict
markov_dict = generate_markov_dict(three_words_count)
markov_dict

{('__BEGIN__', 'おいしい'): {'words': ['ビール'], 'weights': [2]},
 ('おいしい', 'ビール'): {'words': ['を', 'は'], 'weights': [1, 1]},
 ('ビール', 'を'): {'words': ['飲も'], 'weights': [2]},
 ('を', '飲も'): {'words': ['う'], 'weights': [2]},
 ('飲も', 'う'): {'words': ['__END__'], 'weights': [2]},
 ('__BEGIN__', 'ビール'): {'words': ['を'], 'weights': [1]},
 ('ビール', 'は'): {'words': ['生'], 'weights': [1]},
 ('は', '生'): {'words': ['__END__'], 'weights': [1]}}

In [9]:
sentence = 'あえいうえおあお'
d = {}
for char in sentence:
    if char in d:
        d[char] += 1
    else:
        d[char] = 1
d

{'あ': 2, 'え': 2, 'い': 1, 'う': 1, 'お': 2}

In [10]:
from collections import defaultdict
sentence = 'あえいうえおあお'
dd = defaultdict(int)
for char in sentence:
    dd[char] += 1
dd

defaultdict(int, {'あ': 2, 'え': 2, 'い': 1, 'う': 1, 'お': 2})

In [11]:
from collections import defaultdict

def get_first_word_and_count(three_words_count):
    """最初の単語を選択するための辞書データを作成する"""
    first_word_count = defaultdict(int)
    
    for three_words, count in three_words_count.items():
        if three_words[0] == BEGIN:
            next_word = three_words[1]
            first_word_count[next_word] += count
    return first_word_count

get_first_word_and_count(three_words_count)

defaultdict(int, {'おいしい': 2, 'ビール': 1})

In [12]:
def get_first_words_weights(three_words_count):
    """最初の単語と重みのリストを作成する"""
    first_word_count = get_first_word_and_count(three_words_count)
    words = []
    weights = []
    for word, count in first_word_count.items():
        words.append(word)
        weights.append(count)
    
    return words, weights

first_words, first_weights = get_first_words_weights(three_words_count)
first_words, first_weights

(['おいしい', 'ビール'], [2, 1])

In [15]:
import random

def generate_text(first_words, first_weights, markov_dict):
    """入力された辞書データを元に文章を生成する"""
    first_word = random.choices(first_words, weights=first_weights)[0]
    generate_words = [BEGIN, first_word]
    while True:
        pair = tuple(generate_words[-2:])
        words = markov_dict[pair]['words']
        weights = markov_dict[pair]['weights']
        next_word = random.choices(words, weights=weights)[0]
        if next_word == END:
            break
        generate_words.append(next_word)
    return '' .join(generate_words[1:])

In [17]:
for _ in range(5):
    text = generate_text(first_words, first_weights, markov_dict)
    print(text)

おいしいビールを飲もう
ビールを飲もう
おいしいビールは生
おいしいビールを飲もう
おいしいビールを飲もう


In [18]:
import requests

url = 'https://www.aozora.gr.jp/cards/000035/files/301_ruby_5915.zip'
r = requests.get(url)
content = r.content

In [19]:
import io
import zipfile

f = io.BytesIO(content)
zipf = zipfile.ZipFile(f)
namelist = zipf.namelist()
namelist

['ningen_shikkaku.txt']

In [20]:
data = zipf.read(namelist[0])
original_text = data.decode('Shift_JIS')
print(original_text[:500])

人間失格
太宰治

-------------------------------------------------------
【テキスト中に現れる記号について】

《》：ルビ
（例）従姉妹《いとこ》

｜：ルビの付く文字列の始まりを特定する記号
（例）昔｜気質《かたぎ》

［＃］：入力者注　主に外字の説明や、傍点の位置の指定
（例）［＃３字下げ］はしがき［＃「はしがき」は大見出し］
-------------------------------------------------------

［＃３字下げ］はしがき［＃「はしがき」は大見出し］


　私は、その男の写真を三葉、見たことがある。
　一葉は、その男の、幼年時代、とでも言うべきであろうか、十歳前後かと推定される頃の写真であって、その子供が大勢の女のひとに取りかこまれ、（それは、その子供の姉たち、妹たち、それから、従姉妹《いとこ》たちかと想像される）庭園の池のほとりに、荒い縞の袴《はかま》をはいて立ち、首を三十度ほど左に傾け、醜く笑っている写真である。醜く？　けれども、鈍い人たち（


In [23]:
import re

first_sentence = '私は、その男の写真を三葉、見たことがある。'
last_sentence = '神様みたいないい子でした'
_, text = original_text.split(first_sentence)
text, _ = text.split(last_sentence)
text = first_sentence + text + last_sentence

text = text.replace('|', '').replace('　', '')
text = re.sub('《\w+》', '', text)
text = re.sub('［#\w+］', '', text)
text = text.replace('\r', '').replace('\n', '')
text = re.sub('［、「」？］', '', text)
text = re.sub('（\w+）', '', text)
text = re.sub('［\w+］', '', text)

sentences = text.split('。')
print('文の数：', len(sentences))
sentences[:10]


文の数： 1177


['私は、その男の写真を三葉、見たことがある',
 '一葉は、その男の、幼年時代、とでも言うべきであろうか、十歳前後かと推定される頃の写真であって、その子供が大勢の女のひとに取りかこまれ、（それは、その子供の姉たち、妹たち、それから、従姉妹たちかと想像される）庭園の池のほとりに、荒い縞の袴をはいて立ち、首を三十度ほど左に傾け、醜く笑っている写真である',
 '醜く？けれども、鈍い人たち（つまり、美醜などに関心を持たぬ人たち）は、面白くも何とも無いような顔をして、「可愛い坊ちゃんですね」といい加減なお世辞を言っても、まんざら空お世辞に聞えないくらいの、謂わば通俗の「可愛らしさ」みたいな影もその子供の笑顔に無いわけではないのだが、しかし、いささかでも、美醜に就いての訓練を経て来たひとなら、ひとめ見てすぐ、「なんて、いやな子供だ」と頗る不快そうに呟き、毛虫でも払いのける時のような手つきで、その写真をほうり投げるかも知れない',
 'まったく、その子供の笑顔は、よく見れば見るほど、何とも知れず、イヤな薄気味悪いものが感ぜられて来る',
 'どだい、それは、笑顔でない',
 'この子は、少しも笑ってはいないのだ',
 'その証拠には、この子は、両方のこぶしを固く握って立っている',
 '人間は、こぶしを固く握りながら笑えるものでは無いのである',
 '猿だ',
 '猿の笑顔だ']

In [19]:
from tqdm import tqdm

three_words_list = []
for sentence in tqdm(sentences):
    three_words_list += get_three_words_list(sentence)
three_words_count = Counter(three_words_list)
len(three_words_count)

100%|██████████| 1177/1177 [00:55<00:00, 21.08it/s]


32236

In [20]:
markov_dict = generate_markov_dict(three_words_count)
print(len(markov_dict))
first_words, first_weights = get_first_words_weights(three_words_count)
print(len(first_words))

19350
447


In [25]:
for _ in range(5):
    sentence = generate_text(first_words, first_weights, markov_dict)
    print(sentence)

しかも、それから、ロイドの映画がその海と桜の中学校を自分は慄然としてしまったのでした
何という失敗、自分に何の事ではないでしょうか
「お宅は、団体生活というものを食べた記憶は、ほとんど半狂乱になったのであるが、ひび割れてしまったのだ
ただ、みすぼらしい、貧乏くさい女だったと思うように、かえって大カフエのお店に坐っておられなくなるのは、自分の漫画も、くだものも、その用事をいちども覚えようとしたら？罪と祈り、罪の対語は、俗な言い方ですけど）その時、ふとマダムは口調を改め、あなたのほうが風がわりで面白い遊びだから、こんな貧乏くさい女には、最も気のきく」おじさんが遊び相手として附合って救われるのは、素直に今迄のからだもその頃には、自分の若白髪は、いったいどうなんだぜ」こんな、貧乏なのですが、とにかく引受けてくれている人間の女性のほうから、自分はたいてい、おっかなびっくりで、思い切って、そのような権利を留保しても、なかなか治癒し難い傷でした
すぐ近所であったから、思いがけぬ恩を受けて、必ずすぐにそれをこそ挙げるべきだ」彼はその遊戯を当時、ハロルド・ロイドとかいう外国の遊戯場みたいにごまかしながら鮨を食わせる店というところにこそ罪が重なり、苦悩が増大し強烈になるようですが、法律とは言えませんし、ただ神の笞を受ける、そのとおりに、すぐ外出しなければならないのだ」「泣きました


In [26]:
import pickle

with open('markov-dict.pickle', 'wb') as f:
    data = (first_words, first_weights, markov_dict)
    pickle.dump(data, f)